## Selena Data Visualization

Visualisasi arus kas pengeluaran dan pemasukan dari pengguna. File yang didapatkan nanti adalah json dan outputnya adalah visualisasi.

## Import Libraries

In [1]:
import csv
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

## Preprocessing Data

In [10]:
#FUNGSI UNTUK PREPROCESSING DATA

def preprocess_data(data_path):
  # Membaca file dataset
  df = pd.DataFrame(pd.read_csv(data_path))

  # Mengambil kolom yang dibutuhkan
  data_filtered = df[['TGL_TRAN', 'MUTASI_DEBET', 'MUTASI_KREDIT']].copy()

  # Menghapus titik dari kolom 'MUTASI_DEBET'
  #data_filtered['Total Pembayaran'] = data_filtered['Total Pembayaran'].astype(str).str.replace('.', '', regex=False)

  # Mengkonversi tipe data
  data_filtered['MUTASI_DEBET'] = pd.to_numeric(data_filtered['MUTASI_DEBET'])
  data_filtered['MUTASI_KREDIT'] = pd.to_numeric(data_filtered['MUTASI_KREDIT'])
  data_filtered['TGL_TRAN'] = pd.to_datetime(data_filtered['TGL_TRAN']).dt.date

  # Assign value variabel time dan series
  time = data_filtered['TGL_TRAN'].tolist()
  expense = data_filtered['MUTASI_DEBET'].tolist()
  income = data_filtered['MUTASI_KREDIT'].tolist()

  return time, expense, income

In [11]:
### Inisiasi variabel global ###

DATA_PATH = './data/Dataset_BRI.csv'
TIME, EXPENSE, INCOME = preprocess_data(DATA_PATH)

In [ ]:
#Visualisasi Data

plt.plot(TIME, EXPENSE, label = "Pengeluaran")
plt.plot(TIME, INCOME, label = "Pemasukan")
plt.show()